In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 32.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/My Drive/celonis/labelled_text.csv", encoding='ISO-8859-1', header=None)

In [6]:
texts, labels = list(data[data[2] != "Irrelevant"][3]), list(data[data[2] != "Irrelevant"][2])

In [7]:
len(texts), len(labels)

(61692, 61692)

In [8]:
labels_numeric = []
for label in labels:
    if label == "Neutral":
        labels_numeric.append(0)
    elif label == "Positive":
        labels_numeric.append(1)
    else:
        labels_numeric.append(2)

In [9]:
len(texts), len(labels_numeric)

(61692, 61692)

In [10]:
#Clean NaNs
clean_texts, clean_labels = [], []
for i, text in enumerate(texts):
    if isinstance(text, str):
        clean_texts.append(text)
        clean_labels.append(labels_numeric[i])

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [23]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
text_data = []

for text in clean_texts:
    text_data.append(tokenizer.encode(text, return_tensors="tf")[0])

In [14]:
len(clean_texts), len(clean_labels)

(61121, 61121)

In [38]:
final_model = tf.keras.Sequential()
final_model.add(model)
final_model.add(tf.keras.layers.Softmax())

In [48]:
class FinalModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.model = model
    self.softmax = tf.keras.layers.Softmax()
  
  def call(self, inputs):
    out = self.model(inputs)
    return out[0]

In [49]:
final_model = FinalModel()

In [16]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    text_data, padding="post", maxlen=20
)

In [30]:
one_hot_labels = tf.one_hot(clean_labels, depth=3)

In [32]:
#labels_data = tf.convert_to_tensor(clean_labels)
padded_inputs = tf.convert_to_tensor(padded_inputs)

In [33]:
one_hot_labels.shape, padded_inputs.shape

(TensorShape([61121, 3]), TensorShape([61121, 20]))

In [60]:
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [61]:
history = final_model.fit(x=padded_inputs, y=one_hot_labels, batch_size=64, epochs=1)

  7/956 [..............................] - ETA: 3:24:35 - loss: 1.1037 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [64]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=tokenizer.vocab_size,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Softmax()
])

In [66]:
predictions = model1.predict(np.array([padded_inputs[0]]))
print(predictions)

[[0.3374993  0.33312336 0.32937732]]


In [70]:
model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [71]:
history = model1.fit(x=padded_inputs, y=one_hot_labels, epochs=10,
                    validation_steps=30)

Epoch 1/10
1911/1911 [==============================] - 155s 59ms/step - loss: 0.8750 - accuracy: 0.6006
Epoch 2/10
1911/1911 [==============================] - 111s 58ms/step - loss: 0.5803 - accuracy: 0.7740
Epoch 3/10
1911/1911 [==============================] - 119s 63ms/step - loss: 0.4490 - accuracy: 0.8285
Epoch 4/10
1911/1911 [==============================] - 114s 60ms/step - loss: 0.3769 - accuracy: 0.8576
Epoch 5/10
1911/1911 [==============================] - 116s 61ms/step - loss: 0.3271 - accuracy: 0.8767
Epoch 6/10
1911/1911 [==============================] - 115s 60ms/step - loss: 0.2881 - accuracy: 0.8906
Epoch 7/10
1911/1911 [==============================] - 115s 60ms/step - loss: 0.2575 - accuracy: 0.9022
Epoch 8/10
1911/1911 [==============================] - 115s 60ms/step - loss: 0.2332 - accuracy: 0.9107
Epoch 9/10
1911/1911 [==============================] - 116s 61ms/step - loss: 0.2132 - accuracy: 0.9174
Epoch 10/10
1911/1911 [==============================] 

In [72]:
model1.save("/content/drive/My Drive/celonis/lstmv1")

INFO:tensorflow:Assets written to: /content/drive/My Drive/celonis/lstmv1/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/celonis/lstmv1/assets
